# Rotinas de processamento textuais

In [5]:
import numpy as np
import pandas as pd
import nltk
import re
from collections import Counter
import unidecode
import datetime as dt
nltk.download('rslp')
nltk.download('punkt')

arquivo_termos_esg = 'palavras_chave_esg.xlsx'
base_noticias = 'base_noticias.xlsx'

'''
Remove acentos
'''
def remove_acentos(texto):
    return unidecode.unidecode(texto)


'''
Retira espaços e tabs (trailing e dentro)
'''
def trim_texto(texto):
    texto = ''.join(filter(lambda character:ord(character) < 0xff,texto))
    texto = ' '.join(texto.split())
    return texto
    
'''
Remove termos comuns para não pesquisar
'''
def remove_termos_comuns(lista_empresas):
    stopw = ['le', 'paulo','investimentos', 'brasil', 'carlos', 'rede', 'rio', 'ser', 'pao', 'time', 'joao', 'viver', 'rumo', 'oi', 'santos', 'porto', 'soma', 'construtora', 'transmissao', 'blue', 'pague', 'smart', 'log', 'nacional', 'siderurgica', 'mateus', 'cury', 'mundial', 'boa', 'caixa']
    result = lista_empresas
    
    for w in stopw:
        if w in result:
            result.remove(w)
    return result
    
    
'''
Remove nomes compostos (ex Lojas Americanas > Americanas, Banco Itaú > Itaú)
'''
def remove_nome_composto(nome_composto):
    termos_remover = ['são', 'do', 'de', 'da', 'das', 'dos', 'boa', 'indústria', 'indústrias', 'banco','grupo','consórcio', 'construtora', 'comércio', 'atacadista', 'varejista', 'lojas', 'conservas', 'energia', 'paulista','alimentos', 'alimentos','empresa','brasil', 's.a.', 's/a', 'participações', 'br', 'ltda', 'm.' ]
    
    nome = list(nome_composto.lower().split(' '))
    i = 0

    while i < len(termos_remover) and len(nome) > 1:
        if termos_remover[i] in nome:
            nome.remove(termos_remover[i])
        
        i = i+1
    
    return nome[0].split('-')[0]

        
    
'''
Faz o stemming de um texto
'''
def aplica_stemming_texto(texto):
    stemmer = nltk.stem.RSLPStemmer()
    return remove_acentos(' '.join(stemmer.stem(token) for token in nltk.word_tokenize(texto)))


'''
Remove certas palavras de um texto
'''
def remove_palavras_texto(texto, lista_palavras):
    query = texto
    stopwords = lista_palavras

    return ' '.join(  [word for word in re.split("\W+",query) if word.lower() not in stopwords])


'''
 Conta os termos ESG num texto. A fonte dos termos é um Dataframe
'''
def conta_termos_esg(texto, termos):

    texto_stem = aplica_stemming_texto(texto)

    termos['QtdeE'] = 0
    termos['QtdeS'] = 0
    termos['QtdeG'] = 0

    termos['QtdeE'] = termos['E'].apply(lambda x: texto_stem.count(aplica_stemming_texto(x) ) if not pd.isna(x) else 0)
    termos['QtdeS'] = termos['S'].apply(lambda x: texto_stem.count(aplica_stemming_texto(x) ) if not pd.isna(x) else 0)
    termos['QtdeG'] = termos['G'].apply(lambda x: texto_stem.count(aplica_stemming_texto(x) ) if not pd.isna(x) else 0)
    
    r =  ( sum(termos['QtdeE']) , sum(termos['QtdeS']), sum(termos['QtdeG']) )
    
    return r


'''
 Verifica a qual categoria ESG o texto mais se relaciona pela soma de contagem dos termos
'''
def classifica_texto(texto, termos):
    cont = conta_termos_esg(texto, termos)
    if cont[0] > cont[1] and cont[0] > cont[2]:
        return 'E'
    elif cont[1] > cont[0] and cont[1] > cont[2]:
        return 'S'
    elif cont[2] > cont[0] and cont[2] > cont[1]:
        return 'G'
    else:
        return 'Outros'
    
'''
Classifica todas as noticias
'''
def classifica_textos_coletados(noticias):
    dfTermos = pd.read_excel(arquivo_termos_esg)
    
    noticias['classificacao'] = noticias['texto_completo'].apply(lambda x : classifica_texto(x, dfTermos ) )
    
    return noticias


'''
Filtra notícias não relacionadas
'''
def filtra_noticias_nao_relacionadas(noticias, empresa):
    df = noticias
    emp_ajustada = remove_nome_composto(remove_acentos(empresa)).lower()
    df = df[ df['texto_completo'].apply(lambda x : remove_acentos(x.lower())).str.contains(emp_ajustada)   ]
    return df


'''
Filtra notícias sem classificacao
'''
def filtra_noticias_sem_classificacao(noticias, empresa):
    df = noticias
    df = df[df['classificacao'] != 'Outros' ]
    return df


    
'''
 Conta a quatidade de empresas citadas em cada noticia (empresa selecionada x demais)
'''
def conta_mencoes_empresas(noticias, empresa, listagem_empresas):
    df2 = noticias
    df = noticias
    
    emp_ajustada = remove_acentos(remove_nome_composto(empresa.lower()))
    empresas_ajust = list(listagem_empresas['Nome'])
    empresas_ajust = remove_termos_comuns(list(set([remove_acentos(remove_nome_composto(i).lower()) for i in empresas_ajust])))

    if emp_ajustada not in empresas_ajust:
        empresas_ajust.append(emp_ajustada)

    wordlist = list(map(str.lower, empresas_ajust))
    wordlist = list(set(wordlist))
    counters = df2['texto_completo'].apply(lambda t : Counter(re.findall(r'\b[a-z0-9]+\b', remove_palavras_texto(remove_acentos(t.lower()), list(noticias.columns)))))
    
    df2 = pd.concat([df2, counters.apply(pd.Series).fillna(0).astype(int)], axis=1)
    other_words = list(set(df2.columns) - set(wordlist) - set(noticias.columns))
    df2 = df2.drop(other_words, axis=1)

    demais_empresas = list( set(wordlist).intersection(df2.columns))
    
    if emp_ajustada in demais_empresas:
        demais_empresas.remove(emp_ajustada)

    df2['demais_citacoes'] = df2[demais_empresas].sum(axis=1)

    df2 = df2.drop(demais_empresas, axis=1)

    df2.rename(columns = {emp_ajustada:'citacoes_empresa'}, inplace = True)
    
    return df2



'''
Verifica se a citação a empresa é relevante (> soma das demais ou aparecer no titulo)
'''
def filtra_citacao_relevante(noticias, empresa, listagem_empresas, threshold=1.0, aceitar_titulo=True, recalcular_contagem=True):
    df2 = noticias
    if recalcular_contagem:    
        df2 = conta_mencoes_empresas(noticias, empresa, listagem_empresas)
        
    if aceitar_titulo:
        df2['relevante'] = df2.apply(lambda row : 
                                     1 if ( row['citacoes_empresa'] > threshold*row['demais_citacoes']  or remove_acentos(empresa).lower() in remove_acentos(row['titulo'].lower()) )
                                     else 0, axis=1)
    else:
        df2['relevante'] = df2.apply(lambda row : 
                                     1 if ( row['citacoes_empresa'] > threshold*row['demais_citacoes']   )
                                     else 0, axis=1)
    
    
    return df2[ df2['relevante'] == 1 ]


    

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
